In [1]:
from pymongo import ASCENDING
from services.db_service import DbService

pool_collection = DbService().pool_collection()
deck_collection = DbService().deck_collection()

In [2]:
colors = {
    "W": "01-white.txt",
    "U": "02-blue.txt",
    "B": "03-black.txt",
    "R": "04-red.txt",
    "G": "05-green.txt",
    "M": "06-multicolored.txt",
    "C": "07-colorless.txt",
    "L": "08-lands.txt",
}

In [3]:
def get_pipeline(color):
    return [
    { "$match": { "banned": { "$ne": True } } },
    { "$project": { "all_cards": { "$concatArrays": ["$maindeck", "$sideboard"] } } },
    { "$unwind": "$all_cards" },
    { "$group": { "_id": { "_id": "$_id", "name": "$all_cards" }, "count": { "$sum": 1 } } },
    { "$project": { "_id": "$_id._id", "name": "$_id.name", "count": 1 } },
    { "$sort": { "_id": 1 } },
    { "$group": { "_id": "$name", "count": { "$max": "$count" }, "total_decks": { "$sum": 1 } } },
    { "$match": { "total_decks": { "$gte": 25 } } },
    { "$sort": { "_id": 1 } },
    { "$lookup": { "from": "pool", "localField": "_id", "foreignField": "name", "as": "pool" } },
    { "$unwind": "$pool" },
    { "$project": { "_id": "$pool._id", "color": "$pool.color", "name": "$pool.name", "scryfall_name": "$pool.scryfall_name", "count": "$pool.count", "total_decks": 1 } },
    { "$match": { "color": color } },
    { "$sort": { "name": 1 } }
]

In [4]:
for color, file in colors.items():    
    cards = list(deck_collection.aggregate(get_pipeline(color)))
    with open(f"../pool/{file}", "w") as f:
        for card in cards:
            f.write(f"{card['count']} {card['scryfall_name']}\n")